In [3]:
# -*- coding: utf-8 -*-

# /* Import library */
import numpy as np
import os
import shutil
import itertools
import re
import json
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from ttkwidgets import CheckboxTreeview
from ttkwidgets import AutoHideScrollbar
from ttkthemes import *
from psd_tools import PSDImage
from PIL import Image, ImageTk

In [4]:
# /* Constant */
PATH = os.getcwd()
PSD_PATH = os.path.join(PATH, "psd")
DEST_PATH = os.path.join(PATH, "dest")
YMM3 = "-YMM3"
YMM4 = "-YMM4"
QUALITY = 95
BG_IMG = Image.open("./res/alpha_bg.png")

# Array
Y_VER_ARR = np.array(
    [YMM3, YMM4], dtype=object
)
Y_FOLDER_Y3_ARR = np.array(
    ["顔", "後", "口", "全", "他", "体", "髪", "眉", "目"], 
    dtype=object
)
Y_FOLDER_Y4_ARR = np.array(
    ["顔色", "後", "口", "他", "体", "髪", "眉", "目"], 
    dtype=object
)

# Class

In [5]:
# /* Class */
class PSDTool():
    def __init__(self) -> None:
        self.output_path_arr = np.array([], dtype=object)
        self.load_config_json()
    
    def load_config_json(self):
        json_open = open("./cfg/config.json", 'r')
        json_load = json.load(json_open)
        self.folder_paths = json_load["folder_paths"]
    
    def load_psd_file(self, path):
        self.load_psd = PSDImage.open(path)
        self.psd_size = self.load_psd.size
        return self.load_psd
    
    def get_psd_size(self):
        return self.psd_size

In [22]:
class ScrollableFrame(ttk.Frame):
    def __init__(self, parent, size=(1, 1), fit_w=False, fit_h=False):
        """
        parent: tk.Frame
            親となるフレーム
        minimal_size: tuple
            最小キャンバスサイズ
        fit_w: bool
            キャンバス内のフレームの幅をキャンバスに合わせるか否か
        fit_h: bool
            キャンバス内のフレームの高さをキャンバスに合わせるか否か
        """
        ttk.Frame.__init__(self, parent)
        
        # 変数の初期化
        self.minimal_canvas_size = size
        self.fit_width = fit_w
        self.fit_height = fit_h
        
        # 縦スクロールバー
        #vscrollbar = ttk.Scrollbar(self, orient=tk.VERTICAL)
        vscrollbar = AutoHideScrollbar(self, orient=tk.VERTICAL)
        vscrollbar.pack(fill=tk.Y, side=tk.RIGHT, expand=False)
        # 横スクロールバー
        #hscrollbar = ttk.Scrollbar(self, orient=tk.HORIZONTAL)
        hscrollbar = AutoHideScrollbar(self, orient=tk.HORIZONTAL)
        hscrollbar.pack(fill=tk.X, side=tk.BOTTOM, expand=False)
        # Canvas
        self.canvas = tk.Canvas(
            self, bd=0, highlightthickness=0,
            yscrollcommand=vscrollbar.set, xscrollcommand=hscrollbar.set
        )
        self.canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        # スクロールバーを Canvas に関連付け
        vscrollbar.config(command=self.canvas.yview)
        hscrollbar.config(command=self.canvas.xview)
        # Canvas の位置の初期化
        self.canvas.xview_moveto(0)
        self.canvas.yview_moveto(0)
        # スクロール範囲の設定
        self.canvas.config(
            scrollregion=(0, 0, self.minimal_canvas_size[0], self.minimal_canvas_size[1])
        )
        
        # Canvas 内にフレーム作成
        self.interior = ttk.Frame(self.canvas)
        self.interior_id = self.canvas.create_window(
            0, 0, window=self.interior, anchor=tk.NW, 
        )
        
        # 内部フレームの大きさが変わったらCanvasの大きさを変える関数を呼び出す
        self.interior.bind('<Configure>', self._configure_interior)
        self.canvas.bind('<Configure>', self._configure_canvas)
        #self.canvas.bind_all(sequence="<MouseWheel>", func=self._on_mousewheel, add="+")
    
    def bind_child(self, frame=None, sequence=None, func=None, add=None):
        """
        入れ子に bind を設定
        
        Parameters
        ----------
        frame: tk.Frame
            bind を設定するフレーム
        sequuence: str
            イベント内容
        func: function
            イベント内容が実行された場合に呼ばれる関数
        add: str
            一つ前に宣言されるbind関数を実行するのか設定
            "": default, "+"
        """
        children = frame.winfo_children()
        for child in children:
            c_type = type(child)
            if (c_type == tk.Canvas) or (c_type == tk.Frame) or (c_type == ttk.Frame):
                self.bind_child(frame=child, sequence=sequence, func=func, add=add)
            child.bind(sequence=sequence, func=func, add=add)
    
    def update_minimal_canvas_size(self, size):
        """
        Parameters
        ----------
        size: tuple
            最小キャンバスサイズ
        """
        self.minimal_canvas_size = size
    
    def destroy_child(self, frame=None):
        """
        入れ子のウィジェットを削除
        
        Parameters
        ----------
        frame: tk.Frame
            親となるフレーム
        """
        for child in frame.winfo_children():
            child.destroy()
    
    def _configure_interior(self, event):
        """
        Canvas の大きさを変える関数
        
        Parameters
        ----------
        event
            実行される関数の引数へ付与されるイベント情報
        """
        size = (
            max(self.interior.winfo_reqwidth(), self.minimal_canvas_size[0]), 
            max(self.interior.winfo_reqheight(), self.minimal_canvas_size[1])
        )
        self.canvas.config(scrollregion=(0, 0, size[0], size[1]))
        if self.interior.winfo_reqwidth() != self.canvas.winfo_width():
            self.canvas.config(width = self.interior.winfo_reqwidth())
        if self.interior.winfo_reqheight() != self.canvas.winfo_height():
            self.canvas.config(height = self.interior.winfo_reqheight())
    
    def _configure_canvas(self, event):
        """
        Canvas 内のアイテムの大きさを変える関数
        
        Parameters
        ----------
        event
            実行される関数の引数へ付与されるイベント情報
        """
        if (self.interior.winfo_reqwidth() != self.canvas.winfo_width()) and self.fit_width:
            # update the inner frame's width to fill the canvas
            self.canvas.itemconfigure(self.interior_id, width=self.canvas.winfo_width())
        if (self.interior.winfo_reqheight() != self.canvas.winfo_height()) and self.fit_height:
            # update the inner frame's height to fill the canvas
            self.canvas.itemconfigure(self.interior_id, height=self.canvas.winfo_height())
    
    def _on_mousewheel(self, event=None):
        """
        キャンバスの Y スクロールとマウスホイールスクロールを関連付け
        
        Parameters
        ----------
        event
            実行される関数の引数へ付与されるイベント情報
        """
        #if event: self.canvas.yview_scroll(int(-1*(event.delta//120)), 'units')
        if event: self.canvas.yview_scroll(int(-1 * (event.delta / abs(event.delta))), "units")

In [23]:
class App(ThemedTk, PSDTool):
    # 初期化
    def __init__(self, *args, **kwargs):
        # 変数の初期化
        # アルファ背景の読込
        self.bg = BG_IMG
        self.y_folder_list = Y_FOLDER_Y4_ARR
        self.psd_layers = []
        self.psd_layer_imgs = {}
        self.layer_chb_bln = []
        self.layer_chb = []
        
        # tk の初期化
        ThemedTk.__init__(self, *args, **kwargs)
        # テーマの決定
        style = ttk.Style()
        style.theme_use("black")
        #style.configure("Treeview", font=("", 14))
        #style.configure("Checkbutton", height=25)
        # ウィンドウタイトルを決定
        self.title("PSDTool for YMM")
        # ウィンドウの大きさを決定
        self.geometry("1280x720")
        # ウィンドウのグリッドを 1x1 にする
        # この処理をコメントアウトすると配置がズレる
        self.grid_rowconfigure(0, weight=1)
        self.grid_columnconfigure(0, weight=1)
        
        # ページの作成
        self.create_select_page()
        self.create_psd_page()
        self.create_convert_page()
        
        # main_frameを一番上に表示
        self.select_frame.tkraise()
    
    def _change_page(self, page):
        '''
        画面遷移用の関数
        '''
        page.tkraise()
    
    def _set_psd_to_viewer(self):
        """
        読み込んだ psd イメージをビュワーに表示
        """
        for i, layer in enumerate(self.load_psd.descendants()):
            # レイヤーの追加
            self.psd_layers.append(layer)
            if not layer.is_group():
                self.psd_layer_imgs[layer.name] = layer.topil()
        # レイヤーチェックボタンの作成
        self.set_layer_widgets(frame=self.layer_frame.interior)
        # キャンバスサイズの更新
        psd_viewer_size = (
            max(self.psd_size[0], self.psd_viewer_canvas.winfo_width()), 
            max(self.psd_size[1], self.psd_viewer_canvas.winfo_height())
        )
        self.psd_viewer_canvas.config(
            width=psd_viewer_size[0], height=psd_viewer_size[1]
        )
        # プレビューアップデート
        self._update_psd_img()
        # 
        self.psd_viewer.bind_child(frame=self.psd_viewer, sequence="<MouseWheel>", func=self.psd_viewer._on_mousewheel)
        #self.psd_viewer.bind_child(frame=self.psd_viewer.interior, sequence="<MouseWheel>", func=self.psd_viewer._on_mousewheel)
        self.layer_frame.bind_child(frame=self.layer_frame, sequence="<MouseWheel>", func=self.layer_frame._on_mousewheel)
        #self.layer_frame.bind_child(frame=self.layer_frame.interior, sequence="<MouseWheel>", func=self.layer_frame._on_mousewheel)
    
    def _update_psd_img(self):
        """
        psd イメージの更新
        psd のチェックボタンの ON / OFF によって作動する
        """
        self.psd_size = self.get_psd_size()
        # アップデートレイヤーイメージ
        img = self.load_psd.composite()
        self.psd_img = ImageTk.PhotoImage(img)
        # キャンバスにイメージを表示
        self.img_on_psd_viewer_canvas = self.psd_viewer_canvas.create_image(0, 0, image=self.psd_img, anchor=tk.NW)
    
    def _psd_button_clicked(self):
        """
        psd ファイルの読込
        """
        self.ftype = [("Photoshop Data", "*.psd"), ("all files", "*"), ]
        self.psd_file_path = filedialog.askopenfilename(filetypes=self.ftype)
        ext = os.path.splitext(self.psd_file_path)[1]
        if (len(self.psd_file_path) != 0) and (ext != ".psd"):
            tk.messagebox.showerror(
                "ファイル選択エラー", 
                "選択したファイルが .psd ファイルではありません．\n"
            )
        elif (len(self.psd_file_path) != 0) and (ext == ".psd"):
            self.load_psd = self.load_psd_file(path=self.psd_file_path)
            self._set_psd_to_viewer()
            self._change_page(self.psd_frame)
    
    # -----------------------------------select_frame-----------------------------
    
    def create_select_page(self):
        """
        初期画面
        設定画面、変換画面へ移行
        """
        # セレクトページフレーム作成
        self.select_frame = ttk.Frame()
        self.select_frame.grid(row=0, column=0, sticky="nsew")
        # セッティングフレームに移動するボタン
        self.change_psd_button = ttk.Button(
            self.select_frame, 
            text="Setting", 
            command=lambda : self._psd_button_clicked()
        )
        self.change_psd_button.pack(side=tk.LEFT, anchor=tk.CENTER, expand=True)
        # コンバートフレームに移動するボタン
        self.change_convert_button = ttk.Button(
            self.select_frame, 
            text="Convert", 
            command=lambda: self._change_page(self.convert_frame)
        )
        self.change_convert_button.pack(side=tk.RIGHT, anchor=tk.CENTER, expand=True)
    
    #-----------------------------------------------------------------------------
    
    # -----------------------------------psd_frame---------------------------------
    
    def create_psd_page(self):
        """
        設定画面
        PSDTool を参考に作成
        """
        # PSDTool フレーム作成
        self.psd_frame = ttk.Frame()
        self.psd_frame.grid(row=0, column=0, sticky="nsew")
        
        # フレームセッティング
        self.psd_sx = 0.01 * 9/16
        self.psd_sy = 0.01
        self.psd_tab_x = 0.30 - self.psd_sx * 1.5
        self.psd_tab_y = 1.00 - self.psd_sy * 2
        self.psd_pre_con_x = 0.70 - self.psd_sx * 1.5
        self.psd_pre_con_y = 0.08 - self.psd_sy * 1.5
        self.psd_vie_x = 0.70 - self.psd_sx * 1.5
        self.psd_vie_y = 0.92 - self.psd_sy * 1.5
        # タブフレーム
        self.psd_tab_frame = ttk.Frame(self.psd_frame)
        self.psd_tab_frame.place(
            relx=self.psd_sx, rely=self.psd_sy, 
            relwidth=self.psd_tab_x, relheight=self.psd_tab_y,
        )
        # プレビューコントロールフレーム
        self.psd_preview_control_frame = ttk.Frame(self.psd_frame)
        self.psd_preview_control_frame.place(
            relx=self.psd_tab_x+2*self.psd_sx, rely=self.psd_sy, 
            relwidth=self.psd_pre_con_x, relheight=self.psd_pre_con_y,
        )
        # プレビューフレーム
        self.psd_preview_frame = ttk.Frame(self.psd_frame)
        self.psd_preview_frame.place(
            relx=self.psd_tab_x+2*self.psd_sx, rely=self.psd_pre_con_y+2*self.psd_sy, 
            relwidth=self.psd_vie_x, relheight=self.psd_vie_y,
        )
        
        # タブ：レイヤー・プレビュー
        self.set_psd_tab()
        
        # img viewer
        self.set_viewer_control_widgets()
        self.set_psd_viewer()
    
    def set_psd_tab(self):
        """
        タブ配置
        """
        # Notebookウィジェットの作成
        self.notebook = ttk.Notebook(self.psd_tab_frame)
        # ウィジェットの配置
        self.notebook.pack(side=tk.LEFT, expand=True, fill=tk.BOTH)
        # タブの作成
        self.layer_tab_frame = ttk.Frame(self.notebook)
        self.anim_tab_frame = ttk.Frame(self.notebook)
        self.preview_tab_frame = ttk.Frame(self.notebook)
        # notebookにタブを追加
        self.notebook.add(self.layer_tab_frame, text="レイヤー")
        self.notebook.add(self.anim_tab_frame, text="アニメーション")
        self.notebook.add(self.preview_tab_frame, text="プレビュー")
        # /* レイヤータブ */
        self.set_layer_tab_widgets()
        # /* プレビュータブ */
        self.set_preview_tab_widgets()
    
    def set_layer_tab_widgets(self):
        """
        タブ内にチェックボタン用のキャンバスを配置
        キャンバス内にスクロールバーを配置
        """
        self.layer_tab_frame_size = (
            self.psd_tab_frame.winfo_width(), 
            self.psd_tab_frame.winfo_height()
        )
        self.layer_frame = ScrollableFrame(
            parent=self.layer_tab_frame, 
            minimal_canvas_size=self.layer_tab_frame_size,
            fit_w=True, #fit_h=True
        )
        sx = 0.01
        sy = 0.01 * 9/16
        self.layer_frame.place(
            relx=sx, rely=sy, 
            relwidth=1.00-2*sx, relheight=1.00-2*sy, 
        )
        
        self.layer_ddl_frame = ttk.Frame(self.layer_frame.interior)
        self.layer_ddl_frame.pack(side=tk.RIGHT)
        #self.layer_ddl_frame.place(relx=0.70, rely=0.00, relwidth=0.20, relheight=1.00)
        
        # Canvas 内のアイテムの大きさを変える関数
        def _configure_frame(event):
            if self.layer_ddl_frame.winfo_reqheight() != self.layer_frame.canvas.winfo_height():
                self.layer_ddl_frame.config(height = self.layer_frame.canvas.winfo_height())
        
        self.layer_ddl_frame.bind('<Configure>', _configure_frame)
    
    def set_anim_tab_widgets(self):
        """
        """
        pass
    
    def set_preview_tab_widgets(self):
        """
        YMM に対応したフォルダのプレビュー
        """
        pass
    
    def set_viewer_control_widgets(self):
        """
        """
        self.viewer_control_widget_height = 0.036
        sw = 0.01
        dw = 0.10
        img_name_w = 0.20
        viewer_img_fit_x = 0.00
        save_img_name_x = viewer_img_fit_x + dw + 2*sw
        save_img_num_x = save_img_name_x + img_name_w + sw
        save_img_button_x = save_img_num_x + dw + sw
        back_button_x = 1.00
        
        # Fit button
        self.viewer_img_fit_button = ttk.Button(
            self.psd_preview_control_frame, 
            text="Fit", 
        )
        self.viewer_img_fit_button.place(
            relx=viewer_img_fit_x, rely=0.00, 
            relwidth=dw, relheight=1.00,
        )
        # img_name
        self.save_img_name = tk.StringVar()
        self.save_img_name_entry = ttk.Entry(
            self.psd_preview_control_frame,
            textvariable=self.save_img_name,
            width=4
        )
        self.save_img_name_entry.place(
            relx=save_img_name_x, rely=0.00, 
            relwidth=img_name_w, relheight=1.00,
        )
        # img_number
        self.save_img_num = tk.StringVar()
        self.save_img_num_entry = ttk.Entry(
            self.psd_preview_control_frame,
            textvariable=self.save_img_num,
            width=4
        )
        self.save_img_num_entry.place(
            relx=save_img_num_x, rely=0.00, 
            relwidth=dw, relheight=1.00,
        )
        # save button
        self.img_save_button = ttk.Button(
            self.psd_preview_control_frame,
            text=".png",
        )
        self.img_save_button.place(
            relx=save_img_button_x, rely=0.00, 
            relwidth=dw, relheight=1.00,
        )
        # フレーム1 から main フレームに戻るボタン
        self.back_button = ttk.Button(
            self.psd_preview_control_frame, 
            text="Back", 
            command=lambda: self._change_page(self.select_frame)
        )
        self.back_button.place(
            anchor=tk.NE, 
            relx=back_button_x, rely=0.00, 
            relwidth=dw, relheight=1.00,
        )
    
    def set_psd_viewer(self):
        """
        psd イメージのビュワーを設置
        """
        self.psd_preview_frame_size = (
            self.psd_preview_frame.winfo_width(), 
            self.psd_preview_frame.winfo_height()
        )
        self.psd_viewer = ScrollableFrame(
            parent=self.psd_preview_frame, 
            minimal_canvas_size=self.psd_preview_frame_size
        )
        self.psd_viewer.place(relx=0.00, rely=0.00, relheight=1.00, relwidth=1.00)
        # フレーム内に描画用のキャンバス作成
        self.psd_viewer_canvas = tk.Canvas(
            self.psd_viewer.interior, bd=0, highlightthickness=0, 
        )
        self.psd_viewer_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
    
    def set_layer_widgets(self, frame):
        """
        psd レイヤー・グループ用のチェックボタン設置
        """
        self.layer_num = len(self.psd_layers)
        self.space_width = 10
        #self.space_height = 1.00 / self.layer_num
        self.paragraph_level = 0
        self.stack_group = []
        for i, layer in enumerate(self.psd_layers):
            # ウィジェットの追加
            if (self.psd_layers[i].is_group() == True) and (self.psd_layers[i-1].is_group() == False):
                if (self.paragraph_level > 0): self.paragraph_level -= 1
                if len(self.stack_group) >= 1:
                    # チェックボックスの設置
                    self.set_chb(frame=frame, layer=self.stack_group[-1])
                    del self.stack_group[-1]
            if self.psd_layers[i].is_group() == True:
                self.paragraph_level += 1
                self.stack_group.append(layer)
                continue
            # チェックボックスの設置
            self.set_chb(frame=frame, layer=layer)
            if (layer == self.psd_layers[-1]) and (len(self.stack_group) >= 1):
                for group in self.stack_group:
                    if (self.paragraph_level > 0): self.paragraph_level -= 1
                    self.set_chb(frame=frame, layer=group)
    
    def set_chb(self, frame, layer):
        """
        psd レイヤー・グループ用のチェックボタン設置
        """
        # frame
        self.layer_chb_frame = ttk.Frame(frame)
        self.layer_chb_frame.pack(
            side=tk.BOTTOM, expand=True, fill=tk.BOTH, 
        )
        # チェックボックスの設置
        self.layer_chb_bln.append(tk.BooleanVar())
        self.layer_chb_bln[-1].set(layer.is_visible()) # チェックボックスの初期値
        self.layer_chb.append(ttk.Checkbutton(
            self.layer_chb_frame, text=layer.name, #relief=tk.SOLID,
            padding=5, variable=self.layer_chb_bln[-1], 
        ))
        self.layer_chb[-1].pack(
            side=tk.BOTTOM, anchor=tk.W, fill=tk.BOTH, 
            padx=(self.paragraph_level*self.space_width, 0), 
            ipady=5, expand=True
        )
        # コンボボックス：ドロップダウンの追加
        self.layer_ddl = ttk.Combobox(self.layer_chb_frame, values=Y_FOLDER_Y4_ARR, state="readonly")
        #self.layer_ddl.pack(side=tk.BOTTOM, fill=tk.BOTH)
        self.layer_ddl.place(relx=0.60+0.05*self.paragraph_level, rely=0.00, relwidth=0.20, relheight=1.00)
    
    # -----------------------------------------------------------------------------
    
    # -----------------------------------convert_frame---------------------------------
    
    def create_convert_page(self):
        """
        変換画面
        """
        # コンバートフレーム作成
        self.convert_frame = ttk.Frame()
        self.convert_frame.grid(row=0, column=0, sticky="nsew")
        
        # フレームセッティング
        con_sx = 0.01 * 9/16
        con_sy = 0.01
        con_top_x = 0.00 + con_sx
        con_top_y = 0.00 + con_sy
        con_top_w = 1.00 - 2*con_sx
        con_top_h = 0.08 - 1.5*con_sy
        con_bottom_x = 1.00 - con_sx
        con_bottom_y = 1.00 - con_sy
        con_bottom_w = 1.00 - 2*con_sx
        con_bottom_h = 0.10 - 1.5*con_sy
        con_pre_x = 1.00 - con_top_x
        con_pre_y = con_top_h + 2*con_sy
        con_pre_h = 1.00 - con_top_h - con_bottom_h - 4*con_sy
        con_pre_w = (con_pre_h * 9/16) - (1.5*con_sx)
        con_fol_x, con_fol_y = con_top_x, con_pre_y
        con_fol_h = con_pre_h
        con_fol_w = (1.00 - con_pre_w) - 2*(1.5*con_sx)
        
        # トップウィジェットフレーム
        self.convert_top_widget_l_frame = ttk.Frame(self.convert_frame)
        self.convert_top_widget_l_frame.place(
            relx=con_top_x, rely=con_top_y, 
            relwidth=con_fol_w, relheight=con_top_h,
        )
        self.convert_top_widget_r_frame = ttk.Frame(self.convert_frame)
        self.convert_top_widget_r_frame.place(
            relx=con_pre_x, rely=con_top_y, 
            relwidth=con_pre_w, relheight=con_top_h, 
            anchor=tk.NE
        )
        # フォルダーリストフレーム
        self.convert_folder_frame = ttk.Frame(self.convert_frame)
        self.convert_folder_frame.place(
            relx=con_fol_x, rely=con_fol_y, 
            relwidth=con_fol_w, relheight=con_fol_h,
        )
        # フォルダープレビューフレーム
        self.convert_preview_frame = ttk.Frame(self.convert_frame)
        self.convert_preview_frame.place(
            relx=con_pre_x, rely=con_pre_y, 
            relwidth=con_pre_w, relheight=con_pre_h,
            anchor=tk.NE
        )
        # ボトムウィジェットフレーム
        self.convert_bottom_widget_frame = ttk.Frame(self.convert_frame)
        self.convert_bottom_widget_frame.place(
            relx=con_bottom_x, rely=con_bottom_y, 
            relwidth=con_bottom_w, relheight=con_bottom_h, 
            anchor=tk.SE
        )
        
        self.set_convert_widgets()
        self.set_folder_viewer()
        self.set_convert_psd_previewer()
    
    def set_convert_widgets(self):
        """
        
        """
        # ボタン：フォルダーの追加
        self.add_folder_button = ttk.Button(
            self.convert_top_widget_l_frame,
            text="フォルダーの追加",
        )
        self.add_folder_button.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        # ラベル：プレビュー分割
        self.div_label = ttk.Label(
            self.convert_top_widget_r_frame, 
            relief="solid"
        )
        self.div_label.pack(side=tk.LEFT, fill=tk.Y)
        # コンボボックス：プレビュー分割
        self.div_values = ["1", "2", "3", "4"]
        self.div_combobox = ttk.Combobox(
            self.convert_top_widget_r_frame, 
            width=5, values=self.div_values, state="readonly"
        )
        self.div_combobox.pack(side=tk.LEFT, fill=tk.Y)
        self.div_combobox.set(self.div_values[1])
        # ボタン：セレクトページに戻る
        self.con_back_button = ttk.Button(
            self.convert_top_widget_r_frame,
            text="Back", 
            command= lambda: self._change_page(self.select_frame)
        )
        self.con_back_button.pack(side=tk.RIGHT, fill=tk.Y)
        # ボタン：全変換
        self.convert_button = ttk.Button(
            self.convert_bottom_widget_frame,
            text="全ての psd ファイルの変換．All psd files conversion.",
        )
        self.convert_button.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
    
    def set_folder_viewer(self):
        """
        変換する psd フォルダ群を表示
        キャンバス・スクロールバーの設置
        """
        self.convert_folder_frame_size = (
            self.convert_folder_frame.winfo_width(), 
            self.convert_folder_frame.winfo_height()
        )
        self.folder_list_frame = ScrollableFrame(
            parent=self.convert_folder_frame, 
            minimal_canvas_size=self.convert_folder_frame_size
        )
        self.folder_list_frame.place(
            relx=0.00, rely=0.00, relheight=1.00, relwidth=1.00
        )
    
    def set_convert_psd_previewer(self):
        """
        選択されているフォルダの psd イメージ一覧を表示
        キャンバス・スクロールバーの設置
        """
        self.convert_preview_frame_size = (
            self.convert_preview_frame.winfo_width(), 
            self.convert_preview_frame.winfo_height()
        )
        self.convert_psd_previewer = ScrollableFrame(
            parent=self.convert_preview_frame, 
            minimal_canvas_size=self.convert_preview_frame_size
        )
        self.convert_psd_previewer.place(
            relx=0.00, rely=0.00, relheight=1.00, relwidth=1.00
        )
    # ---------------------------------------------------------------------------------

In [24]:
if __name__ == "__main__":
    app = App()
    # Start App
    app.mainloop()

In [4]:
print(type(ttk.Frame()) is ttk.Frame)

True


In [25]:
a = (1, 1)
a = (2, 2)
print(a)

(2, 2)


In [118]:
class App(ThemedTk, PSDTool):
    # 初期化
    def __init__(self, *args, **kwargs):
        # 変数の初期化
        # アルファ背景の読込
        self.bg = BG_IMG
        self.y_folder_list = Y_FOLDER_Y4_ARR
        self.psd_layers = []
        self.psd_layer_imgs = {}
        self.layer_chb_bln = []
        self.layer_chb = []
        
        # tk の初期化
        ThemedTk.__init__(self, *args, **kwargs)
        # テーマの決定
        style = ttk.Style()
        style.theme_use("black")
        #style.configure("Treeview", font=("", 14))
        #style.configure("Checkbutton", height=25)
        # ウィンドウタイトルを決定
        self.title("PSDTool for YMM")
        # ウィンドウの大きさを決定
        self.geometry("1280x720")
        # ウィンドウのグリッドを 1x1 にする
        # この処理をコメントアウトすると配置がズレる
        self.grid_rowconfigure(0, weight=1)
        self.grid_columnconfigure(0, weight=1)
        
        # ページの作成
        self.create_select_page()
        self.create_psd_page()
        self.create_convert_page()
        
        # main_frameを一番上に表示
        self.select_frame.tkraise()
    
    def _change_page(self, page):
        '''
        画面遷移用の関数
        '''
        page.tkraise()
    
    def _set_psd_to_viewer(self):
        """
        読み込んだ psd イメージをビュワーに表示
        """
        for i, layer in enumerate(self.load_psd.descendants()):
            # レイヤーの追加
            self.psd_layers.append(layer)
            if not layer.is_group():
                self.psd_layer_imgs[layer.name] = layer.topil()
        # レイヤーチェックボタンの作成
        self.set_layer_widgets(frame=self.layer_frame.interior)
        # キャンバスサイズの更新
        psd_viewer_size = (
            max(self.psd_size[0], self.psd_viewer_canvas.winfo_width()), 
            max(self.psd_size[1], self.psd_viewer_canvas.winfo_height())
        )
        self.psd_viewer_canvas.config(
            width=psd_viewer_size[0], height=psd_viewer_size[1]
        )
        # プレビューアップデート
        self._update_psd_img()
    
    def _update_psd_img(self):
        """
        psd イメージの更新
        psd のチェックボタンの ON / OFF によって作動する
        """
        self.psd_size = self.get_psd_size()
        # アップデートレイヤーイメージ
        img = self.load_psd.composite()
        self.psd_img = ImageTk.PhotoImage(img)
        # キャンバスにイメージを表示
        self.img_on_psd_viewer_canvas = self.psd_viewer_canvas.create_image(0, 0, image=self.psd_img, anchor=tk.NW)
    
    def _psd_button_clicked(self):
        """
        psd ファイルの読込
        """
        self.ftype = [("Photoshop Data", "*.psd"), ("all files", "*"), ]
        self.psd_file_path = filedialog.askopenfilename(filetypes=self.ftype)
        ext = os.path.splitext(self.psd_file_path)[1]
        if (len(self.psd_file_path) != 0) and (ext != ".psd"):
            tk.messagebox.showerror(
                "ファイル選択エラー", 
                "選択したファイルが .psd ファイルではありません．\n"
            )
        elif (len(self.psd_file_path) != 0) and (ext == ".psd"):
            self.load_psd = self.load_psd_file(path=self.psd_file_path)
            self._set_psd_to_viewer()
            self._change_page(self.psd_frame)
    
    # -----------------------------------select_frame-----------------------------
    
    def create_select_page(self):
        """
        初期画面
        設定画面、変換画面へ移行
        """
        # セレクトページフレーム作成
        self.select_frame = ttk.Frame()
        self.select_frame.grid(row=0, column=0, sticky="nsew")
        # セッティングフレームに移動するボタン
        self.change_psd_button = ttk.Button(
            self.select_frame, 
            text="Setting", 
            command=lambda : self._psd_button_clicked()
        )
        self.change_psd_button.pack(side=tk.LEFT, anchor=tk.CENTER, expand=True)
        # コンバートフレームに移動するボタン
        self.change_convert_button = ttk.Button(
            self.select_frame, 
            text="Convert", 
            command=lambda: self._change_page(self.convert_frame)
        )
        self.change_convert_button.pack(side=tk.RIGHT, anchor=tk.CENTER, expand=True)
    
    #-----------------------------------------------------------------------------
    
    # -----------------------------------psd_frame---------------------------------
    
    def create_psd_page(self):
        """
        設定画面
        PSDTool を参考に作成
        """
        # PSDTool フレーム作成
        self.psd_frame = ttk.Frame()
        self.psd_frame.grid(row=0, column=0, sticky="nsew")
        
        # フレームセッティング
        self.psd_sx = 0.01 * 9/16
        self.psd_sy = 0.01
        self.psd_tab_x = 0.30 - self.psd_sx * 1.5
        self.psd_tab_y = 1.00 - self.psd_sy * 2
        self.psd_pre_con_x = 0.70 - self.psd_sx * 1.5
        self.psd_pre_con_y = 0.08 - self.psd_sy * 1.5
        self.psd_vie_x = 0.70 - self.psd_sx * 1.5
        self.psd_vie_y = 0.92 - self.psd_sy * 1.5
        # タブフレーム
        self.psd_tab_frame = ttk.Frame(self.psd_frame)
        self.psd_tab_frame.place(
            relx=self.psd_sx, rely=self.psd_sy, 
            relwidth=self.psd_tab_x, relheight=self.psd_tab_y,
        )
        # プレビューコントロールフレーム
        self.psd_preview_control_frame = ttk.Frame(self.psd_frame)
        self.psd_preview_control_frame.place(
            relx=self.psd_tab_x+2*self.psd_sx, rely=self.psd_sy, 
            relwidth=self.psd_pre_con_x, relheight=self.psd_pre_con_y,
        )
        # プレビューフレーム
        self.psd_preview_frame = ttk.Frame(self.psd_frame)
        self.psd_preview_frame.place(
            relx=self.psd_tab_x+2*self.psd_sx, rely=self.psd_pre_con_y+2*self.psd_sy, 
            relwidth=self.psd_vie_x, relheight=self.psd_vie_y,
        )
        
        # タブ：レイヤー・プレビュー
        self.set_psd_tab()
        
        # img viewer
        self.set_viewer_control_widgets()
        self.set_psd_viewer()
    
    def set_psd_tab(self):
        """
        タブ配置
        """
        # Notebookウィジェットの作成
        self.notebook = ttk.Notebook(self.psd_tab_frame)
        # ウィジェットの配置
        self.notebook.pack(side=tk.LEFT, expand=True, fill=tk.BOTH)
        # タブの作成
        self.layer_tab_frame = ttk.Frame(self.notebook)
        self.anim_tab_frame = ttk.Frame(self.notebook)
        self.preview_tab_frame = ttk.Frame(self.notebook)
        # notebookにタブを追加
        self.notebook.add(self.layer_tab_frame, text="レイヤー")
        self.notebook.add(self.anim_tab_frame, text="アニメーション")
        self.notebook.add(self.preview_tab_frame, text="プレビュー")
        # /* レイヤータブ */
        self.set_layer_tab_widgets()
        # /* プレビュータブ */
        self.set_preview_tab_widgets()
    
    def set_layer_tab_widgets(self):
        """
        タブ内にチェックボタン用のキャンバスを配置
        キャンバス内にスクロールバーを配置
        """
        self.layer_tab_frame_size = (
            self.psd_tab_frame.winfo_width(), 
            self.psd_tab_frame.winfo_height()
        )
        self.layer_frame = ScrollableFrame(
            parent=self.layer_tab_frame, 
            minimal_canvas_size=self.layer_tab_frame_size,
            #fit_w=True, fit_h=True
        )
        sx = 0.01
        sy = 0.01 * 9/16
        self.layer_frame.place(
            relx=sx, rely=sy, 
            relwidth=1.00-2*sx, relheight=1.00-2*sy, 
        )
        
        self.layer_ddl_frame = ttk.Frame(self.layer_frame.interior)
        self.layer_ddl_frame.pack(side=tk.RIGHT)
        #self.layer_ddl_frame.place(relx=0.70, rely=0.00, relwidth=0.20, relheight=1.00)
        
        # Canvas 内のアイテムの大きさを変える関数
        def _configure_frame(event):
            if self.layer_ddl_frame.winfo_reqheight() != self.layer_frame.canvas.winfo_height():
                self.layer_ddl_frame.config(height = self.layer_frame.canvas.winfo_height())
        
        self.layer_ddl_frame.bind('<Configure>', _configure_frame)
    
    def set_anim_tab_widgets(self):
        """
        """
        pass
    
    def set_preview_tab_widgets(self):
        """
        YMM に対応したフォルダのプレビュー
        """
        pass
    
    def set_viewer_control_widgets(self):
        """
        """
        self.viewer_control_widget_height = 0.036
        sw = 0.01
        dw = 0.10
        img_name_w = 0.20
        viewer_img_fit_x = 0.00
        save_img_name_x = viewer_img_fit_x + dw + 2*sw
        save_img_num_x = save_img_name_x + img_name_w + sw
        save_img_button_x = save_img_num_x + dw + sw
        back_button_x = 1.00
        
        # Fit button
        self.viewer_img_fit_button = ttk.Button(
            self.psd_preview_control_frame, 
            text="Fit", 
        )
        self.viewer_img_fit_button.place(
            relx=viewer_img_fit_x, rely=0.00, 
            relwidth=dw, relheight=1.00,
        )
        # img_name
        self.save_img_name = tk.StringVar()
        self.save_img_name_entry = ttk.Entry(
            self.psd_preview_control_frame,
            textvariable=self.save_img_name,
            width=4
        )
        self.save_img_name_entry.place(
            relx=save_img_name_x, rely=0.00, 
            relwidth=img_name_w, relheight=1.00,
        )
        # img_number
        self.save_img_num = tk.StringVar()
        self.save_img_num_entry = ttk.Entry(
            self.psd_preview_control_frame,
            textvariable=self.save_img_num,
            width=4
        )
        self.save_img_num_entry.place(
            relx=save_img_num_x, rely=0.00, 
            relwidth=dw, relheight=1.00,
        )
        # save button
        self.img_save_button = ttk.Button(
            self.psd_preview_control_frame,
            text=".png",
        )
        self.img_save_button.place(
            relx=save_img_button_x, rely=0.00, 
            relwidth=dw, relheight=1.00,
        )
        # フレーム1 から main フレームに戻るボタン
        self.back_button = ttk.Button(
            self.psd_preview_control_frame, 
            text="Back", 
            command=lambda: self._change_page(self.select_frame)
        )
        self.back_button.place(
            anchor=tk.NE, 
            relx=back_button_x, rely=0.00, 
            relwidth=dw, relheight=1.00,
        )
    
    def set_psd_viewer(self):
        """
        psd イメージのビュワーを設置
        """
        self.psd_preview_frame_size = (
            self.psd_preview_frame.winfo_width(), 
            self.psd_preview_frame.winfo_height()
        )
        self.psd_viewer = ScrollableFrame(
            parent=self.psd_preview_frame, 
            minimal_canvas_size=self.psd_preview_frame_size
        )
        self.psd_viewer.place(relx=0.00, rely=0.00, relheight=1.00, relwidth=1.00)
        # フレーム内に描画用のキャンバス作成
        self.psd_viewer_canvas = tk.Canvas(
            self.psd_viewer.interior, bd=0, highlightthickness=0, 
        )
        self.psd_viewer_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
    
    def set_layer_widgets(self, frame):
        """
        psd レイヤー・グループ用のチェックボタン設置
        """
        self.layer_num = len(self.psd_layers)
        self.space_width = 10
        #self.space_height = 1.00 / self.layer_num
        self.paragraph_level = 0
        self.stack_group = []
        self.layer_ct = CheckboxTreeview(self.layer_frame.interior, show='tree')
        self.layer_ct.pack(side=tk.BOTTOM, expand=True, fill=tk.BOTH)
        # ウィジェットの追加
        for i, layer in enumerate(self.psd_layers):
            # チェックボックスの設置
            if (layer.is_group()) and (not self.psd_layers[i-1].is_group()) and (self.paragraph_level > 0): 
                self.paragraph_level -= 1
                del self.stack_group[-1]
            
            self.set_chb(frame=frame, layer=layer)
            
            if layer.is_group():
                self.paragraph_level += 1
                self.stack_group.append(layer)
    
    def set_chb(self, frame, layer):
        """
        psd レイヤー・グループ用のチェックボタン設置
        """
        # frame
        #self.layer_chb_frame = ttk.Frame(frame)
        #self.layer_chb_frame.pack(
        #    side=tk.BOTTOM, expand=True, fill=tk.BOTH, 
        #)
        # チェックボックスの設置
        #self.layer_chb_bln.append(tk.BooleanVar())
        #self.layer_chb_bln[-1].set(layer.is_visible()) # チェックボックスの初期値
        layer_iid = str(layer.parent) + layer.name
        if self.paragraph_level == 0:
            self.layer_ct.insert(
                parent="", index="0", iid=layer_iid, 
                text=layer.name, open=True, 
            )
        else:
            parent_iid = str(self.stack_group[-1].parent) + self.stack_group[-1].name
            self.layer_ct.insert(
                parent=parent_iid, index="0", iid=layer_iid, 
                text=layer.name, open=True, 
            )
        if layer.is_visible():
            self.layer_ct.change_state(item=layer_iid, state="checked")
        else:
            self.layer_ct.change_state(item=layer_iid, state="unchecked")
        # コンボボックス：ドロップダウンの追加
        self.layer_ddl = ttk.Combobox(self.layer_ddl_frame, values=Y_FOLDER_Y4_ARR, state="readonly")
        self.layer_ddl.pack(side=tk.BOTTOM, fill=tk.BOTH)
        #self.layer_ddl.place(relx=0.60+0.05*self.paragraph_level, rely=0.00, relwidth=0.15, relheight=1.00)
    
    # -----------------------------------------------------------------------------
    
    # -----------------------------------convert_frame---------------------------------
    
    def create_convert_page(self):
        """
        変換画面
        """
        # コンバートフレーム作成
        self.convert_frame = ttk.Frame()
        self.convert_frame.grid(row=0, column=0, sticky="nsew")
        
        # フレームセッティング
        con_sx = 0.01 * 9/16
        con_sy = 0.01
        con_top_x = 0.00 + con_sx
        con_top_y = 0.00 + con_sy
        con_top_w = 1.00 - 2*con_sx
        con_top_h = 0.08 - 1.5*con_sy
        con_bottom_x = 1.00 - con_sx
        con_bottom_y = 1.00 - con_sy
        con_bottom_w = 1.00 - 2*con_sx
        con_bottom_h = 0.10 - 1.5*con_sy
        con_pre_x = 1.00 - con_top_x
        con_pre_y = con_top_h + 2*con_sy
        con_pre_h = 1.00 - con_top_h - con_bottom_h - 4*con_sy
        con_pre_w = (con_pre_h * 9/16) - (1.5*con_sx)
        con_fol_x, con_fol_y = con_top_x, con_pre_y
        con_fol_h = con_pre_h
        con_fol_w = (1.00 - con_pre_w) - 2*(1.5*con_sx)
        
        # トップウィジェットフレーム
        self.convert_top_widget_l_frame = ttk.Frame(self.convert_frame)
        self.convert_top_widget_l_frame.place(
            relx=con_top_x, rely=con_top_y, 
            relwidth=con_fol_w, relheight=con_top_h,
        )
        self.convert_top_widget_r_frame = ttk.Frame(self.convert_frame)
        self.convert_top_widget_r_frame.place(
            relx=con_pre_x, rely=con_top_y, 
            relwidth=con_pre_w, relheight=con_top_h, 
            anchor=tk.NE
        )
        # フォルダーリストフレーム
        self.convert_folder_frame = ttk.Frame(self.convert_frame)
        self.convert_folder_frame.place(
            relx=con_fol_x, rely=con_fol_y, 
            relwidth=con_fol_w, relheight=con_fol_h,
        )
        # フォルダープレビューフレーム
        self.convert_preview_frame = ttk.Frame(self.convert_frame)
        self.convert_preview_frame.place(
            relx=con_pre_x, rely=con_pre_y, 
            relwidth=con_pre_w, relheight=con_pre_h,
            anchor=tk.NE
        )
        # ボトムウィジェットフレーム
        self.convert_bottom_widget_frame = ttk.Frame(self.convert_frame)
        self.convert_bottom_widget_frame.place(
            relx=con_bottom_x, rely=con_bottom_y, 
            relwidth=con_bottom_w, relheight=con_bottom_h, 
            anchor=tk.SE
        )
        
        self.set_convert_widgets()
        self.set_folder_viewer()
        self.set_convert_psd_previewer()
    
    def set_convert_widgets(self):
        """
        
        """
        # ボタン：フォルダーの追加
        self.add_folder_button = ttk.Button(
            self.convert_top_widget_l_frame,
            text="フォルダーの追加",
        )
        self.add_folder_button.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        # ラベル：プレビュー分割
        self.div_label = ttk.Label(
            self.convert_top_widget_r_frame, 
            relief="solid"
        )
        self.div_label.pack(side=tk.LEFT, fill=tk.Y)
        # コンボボックス：プレビュー分割
        self.div_values = ["1", "2", "3", "4"]
        self.div_combobox = ttk.Combobox(
            self.convert_top_widget_r_frame, 
            width=5, values=self.div_values, state="readonly"
        )
        self.div_combobox.pack(side=tk.LEFT, fill=tk.Y)
        self.div_combobox.set(self.div_values[1])
        # ボタン：セレクトページに戻る
        self.con_back_button = ttk.Button(
            self.convert_top_widget_r_frame,
            text="Back", 
            command= lambda: self._change_page(self.select_frame)
        )
        self.con_back_button.pack(side=tk.RIGHT, fill=tk.Y)
        # ボタン：全変換
        self.convert_button = ttk.Button(
            self.convert_bottom_widget_frame,
            text="全ての psd ファイルの変換．All psd files conversion.",
        )
        self.convert_button.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
    
    def set_folder_viewer(self):
        """
        変換する psd フォルダ群を表示
        キャンバス・スクロールバーの設置
        """
        self.convert_folder_frame_size = (
            self.convert_folder_frame.winfo_width(), 
            self.convert_folder_frame.winfo_height()
        )
        self.folder_list_frame = ScrollableFrame(
            parent=self.convert_folder_frame, 
            minimal_canvas_size=self.convert_folder_frame_size
        )
        self.folder_list_frame.place(
            relx=0.00, rely=0.00, relheight=1.00, relwidth=1.00
        )
    
    def set_convert_psd_previewer(self):
        """
        選択されているフォルダの psd イメージ一覧を表示
        キャンバス・スクロールバーの設置
        """
        self.convert_preview_frame_size = (
            self.convert_preview_frame.winfo_width(), 
            self.convert_preview_frame.winfo_height()
        )
        self.convert_psd_previewer = ScrollableFrame(
            parent=self.convert_preview_frame, 
            minimal_canvas_size=self.convert_preview_frame_size
        )
        self.convert_psd_previewer.place(
            relx=0.00, rely=0.00, relheight=1.00, relwidth=1.00
        )
    # ---------------------------------------------------------------------------------

In [120]:
if __name__ == "__main__":
    app = App()
    # Start App
    app.mainloop()

In [105]:
# -*- coding: utf-8 -*-

# Copyright (c) Juliette Monsel 2018
# For license see LICENSE

from ttkwidgets.autocomplete import AutocompleteCombobox
try:
    import Tkinter as tk
except ImportError:
    import tkinter as tk

window = tk.Tk()
tk.Label(window, text="Combobox with autocompletion for the Tk instance's methods:").pack(side='left')
entry = AutocompleteCombobox(window, width=20, completevalues=dir(window))
entry.pack(side='right')
window.mainloop()

In [72]:
# -*- coding: utf-8 -*-

# Copyright (c) RedFantom 2017
# For license see LICENSE
from ttkwidgets import ItemsCanvas
import tkinter as tk
from tkinter import ttk


root = tk.Tk()

canvas = ItemsCanvas(root)
canvas.pack()

canvas.add_item("Example", font=("default", 13, "italic"), backgroundcolor="green", textcolor="darkblue",
                highlightcolor="blue")

root.mainloop()

In [51]:
# -*- coding: utf-8 -*-

# Copyright (c) Juliette Monsel 2018
# For license see LICENSE

from ttkwidgets import AutoHideScrollbar
import tkinter as tk

window = tk.Tk()
listbox = tk.Listbox(window, height=5)
scrollbar = AutoHideScrollbar(window, command=listbox.yview)
listbox.configure(yscrollcommand=scrollbar.set)

for i in range(10):
    listbox.insert('end', 'item %i' % i)

tk.Label(window, text="Increase the window's height\nto make the scrollbar vanish.").pack(side='top', padx=4, pady=4)
scrollbar.pack(side='right', fill='y')
listbox.pack(side='left', fill='both', expand=True)

window.mainloop()

In [30]:
# -*- coding: utf-8 -*-
"""
import tkinter as tk
from tkinter import ttk
from ttkthemes import *
import PIL.Image, PIL.ImageTk
"""

# /* Class */
class PSDTool():
    def __init__(self) -> None:
        self.output_path_arr = np.array([], dtype=object)
        self.load_config_json()
    
    def load_config_json(self):
        json_open = open("./cfg/config.json", 'r')
        json_load = json.load(json_open)
        self.folder_paths = json_load["folder_paths"]
    
    def load_psd_file(self, path):
        self.load_psd = PSDImage.open(path)
        self.psd_size = self.load_psd.size
        return self.load_psd
    
    def get_psd_size(self):
        return self.psd_size

class ScrollableFrame(ttk.Frame):
    def __init__(self, parent, minimal_canvas_size):
        ttk.Frame.__init__(self, parent)
        
        # 変数の初期化
        self.minimal_canvas_size = minimal_canvas_size
        
        # 縦スクロールバー
        vscrollbar = ttk.Scrollbar(self, orient=tk.VERTICAL)
        #vscrollbar.place(relx=0.98, rely=0.00, relwidth=0.02, relheight=1.00)
        vscrollbar.pack(fill=tk.Y, side=tk.RIGHT, expand=False)
        # 横スクロールバー
        hscrollbar = ttk.Scrollbar(self, orient=tk.HORIZONTAL)
        #hscrollbar.place(relx=0.00, rely=0.98, relwidth=0.98, relheight=0.02,)
        hscrollbar.pack(fill=tk.X, side=tk.BOTTOM, expand=False)
        # Canvas
        self.bg_canvas = tk.Canvas(
            self, bd=0, highlightthickness=0,
            yscrollcommand=vscrollbar.set, xscrollcommand=hscrollbar.set
        )
        #self.bg_canvas.place(relx=0.00, rely=0.00, relwidth=0.98, relheight=0.98)
        self.bg_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        # スクロールバーをCanvasに関連付け
        vscrollbar.config(command=self.bg_canvas.yview)
        hscrollbar.config(command=self.bg_canvas.xview)
        # Canvasの位置の初期化
        self.bg_canvas.xview_moveto(0)
        self.bg_canvas.yview_moveto(0)
        # スクロール範囲の設定
        self.bg_canvas.config(
            scrollregion=(0, 0, self.minimal_canvas_size[0], self.minimal_canvas_size[1])
        )
        self.update()
        # Canvas内にフレーム作成
        self.interior = ttk.Frame(self.bg_canvas)
        self.bg_canvas.create_window(
            0, 0, window=self.interior, anchor=tk.NW, 
            width=7680, 
            #height=self.minimal_canvas_size[1], 
        )
        
        # Canvasの大きさを変える関数
        def _configure_interior(event):
            size = (
                max(self.interior.winfo_reqwidth(), self.minimal_canvas_size[0]), 
                max(self.interior.winfo_reqheight(), self.minimal_canvas_size[1])
            )
            self.bg_canvas.config(scrollregion=(0, 0, size[0], size[1]))
            if self.interior.winfo_reqwidth() != self.bg_canvas.winfo_width():
                self.bg_canvas.config(width = self.interior.winfo_reqwidth())
            if self.interior.winfo_reqheight() != self.bg_canvas.winfo_height():
                self.bg_canvas.config(height = self.interior.winfo_reqheight())
        
        # 内部フレームの大きさが変わったらCanvasの大きさを変える関数を呼び出す
        self.interior.bind('<Configure>', _configure_interior)
    
    def update_minimal_canvas_size(self, size):
        self.minimal_canvas_size = size

class App(ThemedTk, PSDTool):
    # 初期化
    def __init__(self, *args, **kwargs):
        # 変数の初期化
        # アルファ背景の読込
        self.bg = BG_IMG
        self.y_folder_list = Y_FOLDER_Y4_ARR
        self.psd_layers = []
        self.psd_layer_imgs = {}
        self.layer_chb_bln = []
        self.layer_chb = []
        
        # tk の初期化
        ThemedTk.__init__(self, *args, **kwargs)
        # テーマの決定
        style = ttk.Style()
        style.theme_use("black")
        # ウィンドウタイトルを決定
        self.title("PSDTool for YMM")
        # ウィンドウの大きさを決定
        self.geometry("1280x720")
        # ウィンドウのグリッドを 1x1 にする
        # この処理をコメントアウトすると配置がズレる
        self.grid_rowconfigure(0, weight=1)
        self.grid_columnconfigure(0, weight=1)
        
        # ページの作成
        self.create_select_page()
        self.create_psd_page()
        self.create_convert_page()
        
        # main_frameを一番上に表示
        self.select_frame.tkraise()
    
    def _change_page(self, page):
        '''
        画面遷移用の関数
        '''
        page.tkraise()
    
    def _set_psd_to_viewer(self):
        """
        読み込んだ psd イメージをビュワーに表示
        """
        for i, layer in enumerate(self.load_psd.descendants()):
            # レイヤーの追加
            self.psd_layers.append(layer)
            if not layer.is_group():
                self.psd_layer_imgs[layer.name] = layer.topil()
        # レイヤーチェックボタンの作成
        self.set_layer_widgets(chb_frame=self.layer_frame.interior, ddl_frame=self.layer_ddl_frame)
        # キャンバスサイズの更新
        psd_viewer_size = (
            max(self.psd_size[0], self.psd_viewer_canvas.winfo_reqwidth()), 
            max(self.psd_size[1], self.psd_viewer_canvas.winfo_reqheight())
        )
        self.psd_viewer_canvas.config(
            width=psd_viewer_size[0], height=psd_viewer_size[1]
        )
        # プレビューアップデート
        self._update_psd_img()
    
    def _update_psd_img(self):
        """
        psd イメージの更新
        psd のチェックボタンの ON / OFF によって作動する
        """
        self.psd_size = self.get_psd_size()
        # アップデートレイヤーイメージ
        img = self.load_psd.composite()
        self.psd_img = ImageTk.PhotoImage(img)
        # キャンバスにイメージを表示
        self.img_on_psd_viewer_canvas = self.psd_viewer_canvas.create_image(0, 0, image=self.psd_img, anchor=tk.NW)
    
    def _psd_button_clicked(self):
        """
        psd ファイルの読込
        """
        self.ftype = [("Photoshop Data", "*.psd"), ("all files", "*"), ]
        self.psd_file_path = filedialog.askopenfilename(filetypes=self.ftype)
        ext = os.path.splitext(self.psd_file_path)[1]
        if (len(self.psd_file_path) != 0) and (ext != ".psd"):
            tk.messagebox.showerror(
                "ファイル選択エラー", 
                "選択したファイルが .psd ファイルではありません．\n"
            )
        elif (len(self.psd_file_path) != 0) and (ext == ".psd"):
            self.load_psd = self.load_psd_file(path=self.psd_file_path)
            self._set_psd_to_viewer()
            self._change_page(self.psd_frame)
    
    # -----------------------------------select_frame-----------------------------
    
    def create_select_page(self):
        """
        初期画面
        設定画面、変換画面へ移行
        """
        # セレクトページフレーム作成
        self.select_frame = ttk.Frame()
        self.select_frame.grid(row=0, column=0, sticky="nsew")
        # セッティングフレームに移動するボタン
        self.change_psd_button = ttk.Button(
            self.select_frame, 
            text="Setting", 
            command=lambda : self._psd_button_clicked()
        )
        self.change_psd_button.pack(side=tk.LEFT, anchor=tk.CENTER, expand=True)
        # コンバートフレームに移動するボタン
        self.change_convert_button = ttk.Button(
            self.select_frame, 
            text="Convert", 
            command=lambda: self._change_page(self.convert_frame)
        )
        self.change_convert_button.pack(side=tk.RIGHT, anchor=tk.CENTER, expand=True)
    
    #-----------------------------------------------------------------------------
    
    # -----------------------------------psd_frame---------------------------------
    
    def create_psd_page(self):
        """
        設定画面
        PSDTool を参考に作成
        """
        # PSDTool フレーム作成
        self.psd_frame = ttk.Frame()
        self.psd_frame.grid(row=0, column=0, sticky="nsew")
        # フレームセッティング
        self.psd_sx = 0.01 * 9/16
        self.psd_sy = 0.01
        self.psd_tab_x = 0.30 - self.psd_sx * 1.5
        self.psd_tab_y = 1.00 - self.psd_sy * 2
        self.psd_pre_con_x = 0.70 - self.psd_sx * 1.5
        self.psd_pre_con_y = 0.08 - self.psd_sy * 1.5
        self.psd_vie_x = 0.70 - self.psd_sx * 1.5
        self.psd_vie_y = 0.92 - self.psd_sy * 1.5
        # タブフレーム
        self.psd_tab_frame = ttk.Frame(self.psd_frame)
        self.psd_tab_frame.place(
            relx=self.psd_sx, rely=self.psd_sy, 
            relwidth=self.psd_tab_x, relheight=self.psd_tab_y,
        )
        # プレビューコントロールフレーム
        self.psd_preview_control_frame = ttk.Frame(self.psd_frame)
        self.psd_preview_control_frame.place(
            relx=self.psd_tab_x+2*self.psd_sx, rely=self.psd_sy, 
            relwidth=self.psd_pre_con_x, relheight=self.psd_pre_con_y,
        )
        # プレビューフレーム
        self.psd_preview_frame = ttk.Frame(self.psd_frame)
        self.psd_preview_frame.place(
            relx=self.psd_tab_x+2*self.psd_sx, rely=self.psd_pre_con_y+2*self.psd_sy, 
            relwidth=self.psd_vie_x, relheight=self.psd_vie_y,
        )
        
        # タブ：レイヤー・プレビュー
        self.set_psd_tab()
        
        # img viewer
        self.set_viewer_control_widgets()
        self.set_psd_viewer()
    
    def set_psd_tab(self):
        """
        タブ配置
        """
        # Notebookウィジェットの作成
        self.notebook = ttk.Notebook(self.psd_tab_frame)
        # ウィジェットの配置
        self.notebook.place(relx=0.00, rely=0.00, relheight=1.00, relwidth=1.00,)
        # タブの作成
        self.layer_tab_frame = ttk.Frame(self.notebook)
        self.preview_tab_frame = ttk.Frame(self.notebook)
        # notebookにタブを追加
        self.notebook.add(self.layer_tab_frame, text="レイヤー")
        self.notebook.add(self.preview_tab_frame, text="プレビュー")
        # /* レイヤータブ */
        self.set_layer_tab_widgets()
        # /* プレビュータブ */
        self.set_preview_tab_widgets()
    
    def set_layer_tab_widgets(self):
        """
        タブ内にチェックボタン用のキャンバスを配置
        キャンバス内にスクロールバーを配置
        """
        """
        # スクロール用のキャンバス
        self.layer_canvas = tk.Canvas(
            self.layer_tab_frame, 
            width=1000, height=1000, 
            bg="#1c1e1f", bd=0, 
            scrollregion=(0, 0, 0, 2000), 
            highlightthickness=0
        )
        self.layer_canvas.place(relx=0.01, rely=0.01, relheight=0.98, relwidth=0.98)
        # 縦方向スクロールバー
        self.layer_vbar = ttk.Scrollbar(self.layer_tab_frame, orient=tk.VERTICAL) #縦方向
        self.layer_vbar.place(relx=0.94, rely=0.01, relheight=0.98, relwidth=0.05)
        # スクロールバーの制御を Canvas に通知する処理
        self.layer_vbar.config(command=self.layer_canvas.yview)
        # Canvasの可動域をスクロールバーに通知する処理
        self.layer_canvas.config(yscrollcommand=self.layer_vbar.set)
        # canvas に Frame を配置
        self.chb_frame = ttk.Frame(self.layer_canvas)
        self.ddl_frame = ttk.Frame(self.layer_canvas)
        #self.layer_frame.grid_columnconfigure(0, weight=1) # 列の調整
        self.layer_canvas_width = int(self.layer_canvas.cget('width'))
        self.ddl_width = int(self.layer_canvas_width/4)
        print(self.ddl_width)
        self.layer_canvas.create_window(
            (0, 0), 
            window=self.chb_frame, 
            anchor=tk.NW, 
            width=self.layer_canvas.cget('width')
        )
        self.layer_canvas.create_window(
            (self.layer_canvas_width, 0), 
            window=self.ddl_frame, 
            anchor=tk.NE, 
            width=self.ddl_width
        )
        """
        self.layer_tab_frame_size = (
            self.layer_tab_frame.winfo_reqwidth(), 
            self.layer_tab_frame.winfo_reqheight()
        )
        self.layer_frame = ScrollableFrame(
            parent=self.layer_tab_frame, 
            minimal_canvas_size=self.layer_tab_frame_size
        )
        self.layer_frame.place(relx=0.01, rely=0.01, relheight=0.98, relwidth=0.98)
        # layer_frame update
        layer_frame_size = (
            max(self.layer_frame.winfo_width(), self.layer_frame.interior.winfo_reqwidth()), 
            max(self.layer_frame.winfo_height(), self.layer_frame.interior.winfo_reqheight())
        )
        self.layer_frame.interior.config(
            width=layer_frame_size[0], height=layer_frame_size[1]
        )
        print(layer_frame_size)
        """
        self.layer_chb_frame = ttk.Frame(self.layer_frame.bg_canvas)
        self.layer_chb_frame.place(
            relx=0.00, rely=0.00,
            relwidth=1.00, relheight=1.00, 
            anchor=tk.NW
        )"""
        self.layer_ddl_frame = ttk.Frame(self.layer_frame.interior)
        self.layer_ddl_frame.place(
            relx=0.60, rely=0.00, 
            relwidth=0.30, relheight=1.00, 
            anchor=tk.NW
        )
    
    def set_preview_tab_widgets(self):
        """
        YMM に対応したフォルダのプレビュー
        """
        pass
    
    def set_viewer_control_widgets(self):
        """
        """
        self.viewer_control_widget_height = 0.036
        sw = 0.01
        dw = 0.10
        img_name_w = 0.20
        viewer_img_fit_x = 0.00
        save_img_name_x = viewer_img_fit_x + dw + 2*sw
        save_img_num_x = save_img_name_x + img_name_w + sw
        save_img_button_x = save_img_num_x + dw + sw
        back_button_x = 1.00
        
        # Fit button
        self.viewer_img_fit_button = ttk.Button(
            self.psd_preview_control_frame, 
            text="Fit", 
        )
        self.viewer_img_fit_button.place(
            relx=viewer_img_fit_x, rely=0.00, 
            relwidth=dw, relheight=1.00,
        )
        # img_name
        self.save_img_name = tk.StringVar()
        self.save_img_name_entry = ttk.Entry(
            self.psd_preview_control_frame,
            textvariable=self.save_img_name,
            width=4
        )
        self.save_img_name_entry.place(
            relx=save_img_name_x, rely=0.00, 
            relwidth=img_name_w, relheight=1.00,
        )
        # img_number
        self.save_img_num = tk.StringVar()
        self.save_img_num_entry = ttk.Entry(
            self.psd_preview_control_frame,
            textvariable=self.save_img_num,
            width=4
        )
        self.save_img_num_entry.place(
            relx=save_img_num_x, rely=0.00, 
            relwidth=dw, relheight=1.00,
        )
        # save button
        self.img_save_button = ttk.Button(
            self.psd_preview_control_frame,
            text=".png",
        )
        self.img_save_button.place(
            relx=save_img_button_x, rely=0.00, 
            relwidth=dw, relheight=1.00,
        )
        # フレーム1 から main フレームに戻るボタン
        self.back_button = ttk.Button(
            self.psd_preview_control_frame, 
            text="Back", 
            command=lambda: self._change_page(self.select_frame)
        )
        self.back_button.place(
            anchor=tk.NE, 
            relx=back_button_x, rely=0.00, 
            relwidth=dw, relheight=1.00,
        )
    
    def set_psd_viewer(self):
        """
        psd イメージのビュワーを設置
        """
        """
        # Canvas
        self.psd_viewer_canvas = tk.Canvas(
            self.psd_frame, 
            width=100, height=100, 
            bg="#1c1e1f", bd=0, 
            scrollregion=(0, 0, 3000, 2000)
        )
        self.psd_viewer_canvas.place(
            relx=0.99, 
            rely=self.viewer_control_widget_height+0.02, 
            relheight=1.00-0.04-self.viewer_control_widget_height, 
            relwidth=0.68, 
            anchor=tk.NE
        )"""
        self.psd_preview_frame_size = (
            self.psd_preview_frame.winfo_reqwidth(), 
            self.psd_preview_frame.winfo_reqheight()
        )
        self.psd_viewer = ScrollableFrame(
            parent=self.psd_preview_frame, 
            minimal_canvas_size=self.psd_preview_frame_size
        )
        self.psd_viewer.place(relx=0.00, rely=0.00, relheight=1.00, relwidth=1.00)
        # フレーム内に描画用のキャンバス作成
        self.psd_viewer_canvas = tk.Canvas(
            self.psd_viewer.interior, bd=0, highlightthickness=0, 
        )
        self.psd_viewer_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
    
    def set_layer_widgets(self, chb_frame, ddl_frame):
        """
        psd レイヤー・グループ用のチェックボタン設置
        """
        self.layer_num = len(self.psd_layers)
        self.space_width = 10
        #self.space_height = 1.00 / self.layer_num
        self.paragraph_level = 0
        stack_group = []
        for i, layer in enumerate(self.psd_layers):
            # ウィジェットの追加
            if (self.psd_layers[i].is_group() == True) and (self.psd_layers[i-1].is_group() == False):
                if (self.paragraph_level > 0): self.paragraph_level -= 1
                if len(stack_group) >= 1:
                    # チェックボックスの設置
                    self.set_chb(chb_frame=chb_frame, ddl_frame=ddl_frame, layer=stack_group[-1])
                    del stack_group[-1]
            if self.psd_layers[i].is_group() == True:
                self.paragraph_level += 1
                stack_group.append(layer)
                continue
            # チェックボックスの設置
            self.set_chb(chb_frame=chb_frame, ddl_frame=ddl_frame, layer=layer)
            if (layer == self.psd_layers[-1]) and (len(stack_group) >= 1):
                for group in stack_group:
                    if (self.paragraph_level > 0): self.paragraph_level -= 1
                    self.set_chb(chb_frame=chb_frame, ddl_frame=ddl_frame, layer=layer)
    
    def set_chb(self, chb_frame, ddl_frame, layer):
        """
        psd レイヤー・グループ用のチェックボタン設置
        """
        # チェックボックスの設置
        self.layer_chb_bln.append(tk.BooleanVar())
        self.layer_chb_bln[-1].set(layer.is_visible()) # チェックボックスの初期値
        self.layer_chb.append(ttk.Checkbutton(
            chb_frame, text=layer.name, #relief=tk.SOLID,
            padding=5, variable=self.layer_chb_bln[-1], 
        ))
        self.layer_chb[-1].pack(
            side=tk.BOTTOM, anchor=tk.W, fill=tk.BOTH, 
            padx=(self.paragraph_level*self.space_width, 0), 
            ipady=5, expand=True
        )
        self.layer_ddl = ttk.Combobox(ddl_frame, values=Y_FOLDER_Y4_ARR, state="readonly", )
        self.layer_ddl.pack(side=tk.BOTTOM, anchor=tk.E, expand=True)
    
    # -----------------------------------------------------------------------------
    
    # -----------------------------------convert_frame---------------------------------
    
    def create_convert_page(self):
        """
        変換画面
        """
        # コンバートフレーム作成
        self.convert_frame = ttk.Frame()
        self.convert_frame.grid(row=0, column=0, sticky="nsew")
        
        self.set_convert_widgets()
        self.set_folder_viewer()
        self.set_convert_psd_preview_viewer()
    
    def set_convert_widgets(self):
        """
        
        """
        # ボタン：フォルダーの追加
        self.add_folder_button = ttk.Button(
            self.convert_frame,
            text="フォルダーの追加",
        )
        self.add_folder_button.place(
            relx=0.01, rely=0.01, 
            relheight=0.08, relwidth=1.00-0.03-0.79*9/16, 
            anchor=tk.NW
        )
        # ラベル：プレビュー分割
        self.div_label = ttk.Label(
            self.convert_frame, 
            relief="solid"
        )
        self.div_label.place(
            relx=1.00-0.03-0.79*9/16+0.02, rely=0.01, 
            relheight=0.08, relwidth=0.08*9/16
        )
        # コンボボックス：プレビュー分割
        self.div_values = ["1", "2", "3", "4"]
        self.div_combobox = ttk.Combobox(self.convert_frame, width=5, values=self.div_values, state="readonly")
        self.div_combobox.place(
            relx=1.00-0.03-0.79*9/16+0.07, rely=0.01, 
            relheight=0.08, relwidth=0.08*9/16
        )
        self.div_combobox.set(self.div_values[1])
        # ボタン：セレクトページに戻る
        self.con_back_button = ttk.Button(
            self.convert_frame,
            text="Back", 
            command= lambda: self._change_page(self.select_frame)
        )
        self.con_back_button.place(
            relx=0.99, rely=0.01, 
            relheight=0.08, relwidth=0.08, 
            anchor=tk.NE
        )
        # ボタン：全変換
        self.convert_button = ttk.Button(
            self.convert_frame,
            text="全ての psd ファイルの変換．All psd files conversion.",
        )
        self.convert_button.place(
            relx=0.01, rely=0.99, 
            relheight=0.08, relwidth=0.98, 
            anchor=tk.SW
        )
    
    def set_folder_viewer(self):
        """
        変換する psd フォルダ群を表示
        キャンバス・スクロールバーの設置
        """
        """
        # Canvas
        self.folder_canvas = tk.Canvas(
            self.convert_frame, 
            width=1000, height=1000, 
            bg="#1c1e1f", bd=0, 
            scrollregion=(0, 0, 0, 2000)
        )
        self.folder_canvas.place(
            relx=0.01, rely=0.105, 
            relheight=0.79, relwidth=1.00-0.03-0.79*9/16, 
            anchor=tk.NW
        )
        # 縦方向スクロールバー
        self.folder_vbar = ttk.Scrollbar(self.folder_canvas, orient=tk.VERTICAL) #縦方向
        self.folder_vbar.place(relx=0.97, rely=0.01, relheight=0.98, relwidth=0.02)
        # スクロールバーの制御を Canvas に通知する処理
        self.folder_vbar.config(command=self.folder_canvas.yview)
        # Canvasの可動域をスクロールバーに通知する処理
        self.folder_canvas.config(yscrollcommand=self.folder_vbar.set)
        # canvas に Frame を配置
        self.convert_folder_frame = ttk.Frame(self.folder_canvas)
        self.folder_canvas.create_window(
            (0, 0), 
            window=self.convert_folder_frame, 
            anchor=tk.NW, 
            width=self.layer_canvas.winfo_reqwidth()
        )"""
    
    def set_convert_psd_preview_viewer(self):
        """
        選択されているフォルダの psd イメージ一覧を表示
        キャンバス・スクロールバーの設置
        """
        """
        # Canvas
        self.convert_preview_canvas = tk.Canvas(
            self.convert_frame, 
            width=1000, height=1000, 
            bg="#1c1e1f", bd=0, 
            scrollregion=(0, 0, 0, 2000)
        )
        self.convert_preview_canvas.place(
            relx=0.99, rely=0.105, 
            relheight=0.79, relwidth=0.79*9/16, 
            anchor=tk.NE
        )
        # 縦方向スクロールバー
        self.convert_preview_vbar = ttk.Scrollbar(self.convert_preview_canvas, orient=tk.VERTICAL) #縦方向
        self.convert_preview_vbar.place(relx=0.97, rely=0.01, relheight=0.98, relwidth=0.02)
        # スクロールバーの制御を Canvas に通知する処理
        self.convert_preview_vbar.config(command=self.convert_preview_canvas.yview)
        # Canvasの可動域をスクロールバーに通知する処理
        self.convert_preview_canvas.config(yscrollcommand=self.convert_preview_vbar.set)
        # canvas に Frame を配置
        self.convert_preview_frame = ttk.Frame(self.convert_preview_canvas)
        self.convert_preview_canvas.create_window(
            (0, 0), 
            window=self.convert_preview_frame, 
            anchor=tk.NW, 
            width=self.layer_canvas.winfo_reqwidth()
        )"""
    # ---------------------------------------------------------------------------------

In [40]:
if __name__ == "__main__":
    app = App()
    # Start App
    app.mainloop()

(1, 1)


In [41]:
import tkinter as tk
import tkinter.ttk as ttk
from PIL import Image, ImageTk

class ScrollableFrame(tk.Frame):
    def __init__(self, parent, minimal_canvas_size):
        tk.Frame.__init__(self, parent)
        
        # 変数の初期化
        self.minimal_canvas_size = minimal_canvas_size
        
        # 縦スクロールバー
        vscrollbar = ttk.Scrollbar(self, orient=tk.VERTICAL)
        vscrollbar.pack(fill=tk.Y, side=tk.RIGHT, expand=False)
        # 横スクロールバー
        hscrollbar = ttk.Scrollbar(self, orient=tk.HORIZONTAL)
        hscrollbar.pack(fill=tk.X, side=tk.BOTTOM, expand=False)
        # Canvas
        self.canvas = tk.Canvas(self, bd=0, highlightthickness=0,
            yscrollcommand=vscrollbar.set, xscrollcommand=hscrollbar.set)
        self.canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        # スクロールバーをCanvasに関連付け
        vscrollbar.config(command=self.canvas.yview)
        hscrollbar.config(command=self.canvas.xview)
        # Canvasの位置の初期化
        self.canvas.xview_moveto(0)
        self.canvas.yview_moveto(0)
        # スクロール範囲の設定
        self.canvas.config(scrollregion=(0, 0, self.minimal_canvas_size[0], self.minimal_canvas_size[1]))
        
        # Canvas内にフレーム作成
        self.interior = tk.Frame(self.canvas)
        self.canvas.create_window(0, 0, window=self.interior, anchor='nw')
        
        # Canvasの大きさを変える関数
        def _configure_interior(event):
            size = (
                max(self.interior.winfo_reqwidth(), self.minimal_canvas_size[0]), 
                max(self.interior.winfo_reqheight(), self.minimal_canvas_size[1])
            )
            self.canvas.config(scrollregion=(0, 0, size[0], size[1]))
            if self.interior.winfo_reqwidth() != self.canvas.winfo_width():
                self.canvas.config(width = self.interior.winfo_reqwidth())
            if self.interior.winfo_reqheight() != self.canvas.winfo_height():
                self.canvas.config(height = self.interior.winfo_reqheight())
            
        
        # 内部フレームの大きさが変わったらCanvasの大きさを変える関数を呼び出す
        self.interior.bind('<Configure>', _configure_interior)

class Application(tk.Frame):
    def __init__(self, read_image, master=None):
        super().__init__(master)
        self.master = master
        self.master.title('scrollbar trial')
        self.pack(fill=tk.BOTH, expand=True)
        self.create_widgets()
    
    def create_widgets(self):
        self.canvas_frame = ScrollableFrame(self, minimal_canvas_size)
        self.canvas_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        self.control_frame = tk.Frame(self)
        self.control_frame.pack(side=tk.TOP, fill=tk.Y, expand=False)
        
        self.label_title1 = ttk.Label(self.control_frame, text='Window coordinate')
        self.label_title1.pack()
        self.point_x = tk.StringVar()
        self.point_y = tk.StringVar()
        self.label_x = ttk.Label(self.control_frame, textvariable=self.point_x)
        self.label_x.pack()
        self.label_y = ttk.Label(self.control_frame, textvariable=self.point_y)
        self.label_y.pack()
        self.label_title2 = ttk.Label(self.control_frame, text='Canvas coordinate')
        self.label_title2.pack()
        self.point_xc = tk.StringVar()
        self.point_yc = tk.StringVar()
        self.label_xc = ttk.Label(self.control_frame, textvariable=self.point_xc)
        self.label_xc.pack()
        self.label_yc = ttk.Label(self.control_frame, textvariable=self.point_yc)
        self.label_yc.pack()
        self.canvas_frame.canvas.bind('<ButtonPress-1>', self.pickup_point)
        
        # canvasに画像をセットする
        self.im = ImageTk.PhotoImage(read_image)
        self.canvas_frame.canvas.config(width=read_image.width, height=read_image.height)
        self.canvas_frame.canvas.photo = self.im
        self.canvas_frame.canvas.create_image(0, 0, anchor='nw', image=self.im)
    
    # ポインタの座標を取得する
    def pickup_point(self, event):
        self.point_x.set('x : ' + str(event.x))
        self.point_y.set('y : ' + str(event.y))
        self.point_xc.set('x : ' + str(self.canvas_frame.canvas.canvasx(event.x)))
        self.point_yc.set('y : ' + str(self.canvas_frame.canvas.canvasy(event.y)))
        print(event.x, event.y, self.canvas_frame.canvas.canvasx(event.x), self.canvas_frame.canvas.canvasy(event.y))

read_image = Image.open('test_figure.png')
canvas_width, canvas_height = read_image.size
minimal_canvas_size = read_image.size

# アプリケーション起動
root = tk.Tk()
app = Application(read_image=read_image, master=root)
app.mainloop()

In [32]:
import tkinter as tk
import tkinter.ttk as ttk
from PIL import Image, ImageTk
try:
    from ttkwidgets import AutoHideScrollbar
except ImportError:
    from ttkwidgets import AutoHideScrollbar

class ScrollableFrame(ttk.Frame):
    def __init__(self, parent, size=(1, 1), fit_w=False, fit_h=False):
        """
        parent: tk.Frame
            親となるフレーム
        minimal_size: tuple
            最小キャンバスサイズ
        fit_w: bool
            キャンバス内のフレームの幅をキャンバスに合わせるか否か
        fit_h: bool
            キャンバス内のフレームの高さをキャンバスに合わせるか否か
        """
        ttk.Frame.__init__(self, parent)
        
        # 変数の初期化
        self.minimal_canvas_size = size
        self.fit_width = fit_w
        self.fit_height = fit_h
        
        # 縦スクロールバー
        vscrollbar = AutoHideScrollbar(self, orient=tk.VERTICAL)
        #vscrollbar = ttk.Scrollbar(self, orient=tk.VERTICAL)
        vscrollbar.pack(fill=tk.Y, side=tk.RIGHT, expand=False)
        # 横スクロールバー
        #hscrollbar = AutoHideScrollbar(self, orient=tk.HORIZONTAL)
        hscrollbar = ttk.Scrollbar(self, orient=tk.HORIZONTAL)
        hscrollbar.pack(fill=tk.X, side=tk.BOTTOM, expand=False)
        # Canvas
        self.canvas = tk.Canvas(
            self, bd=0, highlightthickness=0,
            yscrollcommand=vscrollbar.set, xscrollcommand=hscrollbar.set
        )
        self.canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        # スクロールバーを Canvas に関連付け
        vscrollbar.config(command=self.canvas.yview)
        hscrollbar.config(command=self.canvas.xview)
        # Canvas の位置の初期化
        self.canvas.xview_moveto(0)
        self.canvas.yview_moveto(0)
        # スクロール範囲の設定
        self.canvas.config(
            scrollregion=(0, 0, self.minimal_canvas_size[0], self.minimal_canvas_size[1])
        )
        
        # Canvas 内にフレーム作成
        self.interior = ttk.Frame(self.canvas)
        self.interior_id = self.canvas.create_window(
            0, 0, window=self.interior, anchor=tk.NW, 
        )
        
        # 内部フレームの大きさが変わったらCanvasの大きさを変える関数を呼び出す
        self.canvas.bind('<Configure>', self._configure_canvas)
        #self.canvas.bind_all(sequence="<MouseWheel>", func=self._on_mousewheel, add="+")
        self.interior.bind('<Configure>', self._configure_interior)
    
    def bind_child(self, frame=None, sequence=None, func=None, add=None):
        """
        入れ子に bind を設定
        
        Parameters
        ----------
        frame: tk.Frame
            bind を設定するフレーム
        sequuence: str
            イベント内容
        func: function
            イベント内容が実行された場合に呼ばれる関数
        add: str
            一つ前に宣言されるbind関数を実行するのか設定
            "": default, "+"
        """
        children = frame.winfo_children()
        for child in children:
            c_type = type(child)
            if (c_type == tk.Canvas) or (c_type == tk.Frame) or (c_type == ttk.Frame):
                self.bind_child(frame=child, sequence=sequence, func=func, add=add)
            child.bind(sequence=sequence, func=func, add=add)
    
    def update_minimal_canvas_size(self, size):
        """
        Parameters
        ----------
        size: tuple
            最小キャンバスサイズ
        """
        self.minimal_canvas_size = size
    
    def destroy_child(self, frame=None):
        """
        入れ子のウィジェットを削除
        
        Parameters
        ----------
        frame: tk.Frame
            親となるフレーム
        """
        for child in frame.winfo_children():
            child.destroy()
    
    def _configure_interior(self, event):
        """
        Canvas の大きさを変える関数
        
        Parameters
        ----------
        event
            実行される関数の引数へ付与されるイベント情報
        """
        size = (
            max(self.interior.winfo_reqwidth(), self.minimal_canvas_size[0]), 
            max(self.interior.winfo_reqheight(), self.minimal_canvas_size[1])
        )
        self.canvas.config(scrollregion=(0, 0, size[0], size[1]))
        if self.interior.winfo_reqwidth() != self.canvas.winfo_width():
            self.canvas.config(width = self.interior.winfo_reqwidth())
        if self.interior.winfo_reqheight() != self.canvas.winfo_height():
            self.canvas.config(height = self.interior.winfo_reqheight())
    
    def _configure_canvas(self, event):
        """
        Canvas 内のアイテムの大きさを変える関数
        
        Parameters
        ----------
        event
            実行される関数の引数へ付与されるイベント情報
        """
        if (self.interior.winfo_reqwidth() != self.canvas.winfo_width()) and self.fit_width:
            # update the inner frame's width to fill the canvas
            self.canvas.itemconfigure(self.interior_id, width=self.canvas.winfo_width())
        if (self.interior.winfo_reqheight() != self.canvas.winfo_height()) and self.fit_height:
            # update the inner frame's height to fill the canvas
            self.canvas.itemconfigure(self.interior_id, height=self.canvas.winfo_height())
    
    def _on_mousewheel(self, event=None):
        """
        キャンバスの Y スクロールとマウスホイールスクロールを関連付け
        
        Parameters
        ----------
        event
            実行される関数の引数へ付与されるイベント情報
        """
        #if event: self.canvas.yview_scroll(int(-1*(event.delta//120)), 'units')
        if event: self.canvas.yview_scroll(int(-1 * (event.delta / abs(event.delta))), "units")

class App(ttk.Frame):
    def __init__(self, read_image, size, master=None):
        super().__init__(master)
        self.master = master
        self.read_img = read_image
        self.size = size
        self.master.title('scrollbar trial')
        self.pack(fill=tk.BOTH, expand=True)
        self.create_widgets()
    
    def create_widgets(self):
        self.canvas_frame = ScrollableFrame(self, size=self.size)
        self.canvas_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        self.control_frame = ttk.Frame(self)
        self.control_frame.pack(side=tk.TOP, fill=tk.Y, expand=False)
        
        self.label_title1 = ttk.Label(self.control_frame, text='Window coordinate')
        self.label_title1.pack()
        self.point_x = tk.StringVar()
        self.point_y = tk.StringVar()
        self.label_x = ttk.Label(self.control_frame, textvariable=self.point_x)
        self.label_x.pack()
        self.label_y = ttk.Label(self.control_frame, textvariable=self.point_y)
        self.label_y.pack()
        self.label_title2 = ttk.Label(self.control_frame, text='Canvas coordinate')
        self.label_title2.pack()
        self.point_xc = tk.StringVar()
        self.point_yc = tk.StringVar()
        self.label_xc = ttk.Label(self.control_frame, textvariable=self.point_xc)
        self.label_xc.pack()
        self.label_yc = ttk.Label(self.control_frame, textvariable=self.point_yc)
        self.label_yc.pack()
        
        # canvasに画像をセットする
        self.img_canvas = tk.Canvas(self.canvas_frame.interior)
        self.img_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        self.img_canvas.bind('<ButtonPress-1>', self.pickup_point)
        self.im = ImageTk.PhotoImage(self.read_img, master=self.master)
        self.img_canvas.config(width=self.read_img.width, height=self.read_img.height)
        self.img_canvas.photo = self.im
        self.img_canvas.create_image((0, 0), anchor=tk.NW, image=self.im)
    
    # ポインタの座標を取得する
    def pickup_point(self, event):
        self.point_x.set('x : ' + str(event.x))
        self.point_y.set('y : ' + str(event.y))
        self.point_xc.set('x : ' + str(self.img_canvas.canvasx(event.x)))
        self.point_yc.set('y : ' + str(self.img_canvas.canvasy(event.y)))
        print(event.x, event.y, self.img_canvas.canvasx(event.x), self.img_canvas.canvasy(event.y))

if __name__ == "__main__":
    read_image = Image.open("test_figure.png")
    canvas_width, canvas_height = read_image.size
    minimal_canvas_size = read_image.size
    
    # アプリケーション起動
    root = tk.Tk()
    app = App(master=root, read_image=read_image, size=minimal_canvas_size)
    app.mainloop()

114 112 114.0 112.0
128 233 128.0 233.0
216 257 216.0 257.0
222 180 222.0 180.0
103 130 103.0 130.0


In [ ]:
if __name__ == "__main__":
    psd_tool = PSDTool()
    psd_tool.load_config_json()
    """
    app = App()
    # Start App
    app.mainloop()
    """

In [1]:
import tkinter
import tkinter.ttk

root = tkinter.Tk()
root.title('ブログ検索順位取得ツール') #タイトル
root.geometry('500x200') #サイズ

#表用のリスト（あとでExcel読み込みに対応させる）
list_keyword = ['理系夫婦','BabyKumon 効果','Python GIF','fortran 配列','python google','BabyKumon 絵本','VBA 入門 ユーザーフォーム']
list_rank = [4,5,8,7,10,12,35]
list_title = ['xxxxxx','yyyyyy','zzzzzz','ssss','dddd','ffff','gggg']

num_list = len(list_keyword) #リストの数

#Canvas widgetを生成
canvas = tkinter.Canvas(root,width=480,height=150,bg='white') #背景を白に
canvas.grid(row=1,rowspan=num_list,column=0,columnspan=5)     #7行x5列分

#スクロールバー
vbar=tkinter.ttk.Scrollbar(root,orient=tkinter.VERTICAL) #縦方向
vbar.grid(row=1,rowspan=7,column=5,sticky='ns')          #7行分の長さで設置

#スクロールバーの制御をCanvasに通知する処理
vbar.config(command=canvas.yview)

#Canvasの可動域をスクロールバーに通知する処理
canvas.config(yscrollcommand=vbar.set)

#スクロール可動域＜＝これがないと、どこまでもスクロールされてしまう。
sc_hgt=int(150/6*(num_list+1))  #スクロールの縦の範囲　リストの数＋ヘッダー分に
canvas.config(scrollregion=(0,0,500,sc_hgt))

#Frameを作成
frame = tkinter.Frame(canvas,bg='white') #背景を白に

#frameをcanvasに配置
canvas.create_window((0,0),window=frame,anchor=tkinter.NW,width=canvas.cget('width'))   #anchor<=NWで左上に寄せる

#header row=1に設定する文字列 余白は0に
e0=tkinter.Label(frame,width=5,text='select',background='white')
e0.grid(row=1,column=0,padx=0,pady=0,ipadx=0,ipady=0) #0列目

e1=tkinter.Label(frame,width=25,text='keyword',background='white')
e1.grid(row=1,column=1,padx=0,pady=0,ipadx=0,ipady=0) #1列目

e2=tkinter.Label(frame,width=5,text='rank',background='white')
e2.grid(row=1,column=2,padx=0,pady=0,ipadx=0,ipady=0) #2列目

e3=tkinter.Label(frame,width=30,text='title',background='white')
e3.grid(row=1,column=3,padx=0,pady=0,ipadx=0,ipady=0) #3列目

irow = 2
irow0=2
erow=num_list+irow0
while irow < erow:   #リストの数分ループしてLabelとチェックボックスを設置
	#色の設定
	if irow%2==0:
		color='#cdfff7'  #薄い青
	else:
		color='white'
	
	#チェックボックスの設置
	bln=tkinter.BooleanVar()
	bln.set(False)           #チェックボックスの初期値
	c = tkinter.Checkbutton(frame,variable = bln,width=5,text='',background='white')
	c.grid(row=irow,column=0,padx=0,pady=0,ipadx=0,ipady=0)  #0列目
	#検索キーワード
	a1=list_keyword[irow-irow0]
	b1=tkinter.Label(frame,width=25,text=a1,background=color)
	b1.grid(row=irow,column=1,padx=0,pady=0,ipadx=0,ipady=0) #1列目
	#検索順位
	a2=list_rank[irow-irow0]
	b2=tkinter.Label(frame,width=5,text=a2,background=color)
	b2.grid(row=irow,column=2,padx=0,pady=0,ipadx=0,ipady=0) #2列目
	#記事タイトル
	a3=list_title[irow-irow0]
	b3=tkinter.Label(frame,width=30,text=a3,background=color)
	b3.grid(row=irow,column=3,padx=0,pady=0,ipadx=0,ipady=0) #3列目
	
	irow=irow+1
	
#ウィンドウを動かす
root.mainloop()

In [108]:
os.getcwd()

'd:\\GitHub-Home\\00-JupyterHome\\2022\\psd-tool-for-ymm\\tests'

In [3]:
from PIL import Image, ImageDraw

IMG_SIZE = (7680, 4320)
BACKGROUND = (255, 255, 255)
GRAY = (187, 187, 187)

SQUARE_LENGTH = 10
REPEAT_X = int(IMG_SIZE[0] / SQUARE_LENGTH + 1)
REPEAT_Y = int(IMG_SIZE[1] / SQUARE_LENGTH + 1)

im = Image.new('RGB', IMG_SIZE, BACKGROUND)
draw = ImageDraw.Draw(im)

for x in range(1, REPEAT_X):
    for y in range(1, REPEAT_Y):
        if ((x % 2 == 0) and (y % 2 != 0)) or ((x % 2 != 0) and (y % 2 == 0)):
            x0 = x * SQUARE_LENGTH
            y0 = y * SQUARE_LENGTH
            x1 = x0 + SQUARE_LENGTH
            y1 = y0 + SQUARE_LENGTH
            draw.rectangle([(x0, y0), (x1, y1)], fill=GRAY)

im.save("alpha_bg.png", quality=95)

In [34]:
a = np.arange(5)
a[-1]

4